In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
print(os.getcwd())
base_directory = '/content' + '/drive/MyDrive/DLDMD'
lorenz96_directory = base_directory + '/examples/lorenz96/'

/content


In [3]:
os.chdir(base_directory)
import tensorflow as tf
from scipy.integrate import solve_ivp
import pickle
import datetime as dt
import os
import numpy as np
import sys
import DLDMD as dl
import LossDLDMD as lf
import Data as dat
import Training as tr
os.chdir(lorenz96_directory)

In [4]:
# ==============================================================================
# Setup
# ==============================================================================
NUM_SAVES = 1        # Number of times to save the model throughout training
NUM_PLOTS = 1        # Number of diagnostic plots to generate while training
DEVICE = '/GPU:0'
GPUS = tf.config.experimental.list_physical_devices('GPU')
if GPUS:
    try:
        for gpu in GPUS:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)
else:
    DEVICE = '/CPU:0'

tf.keras.backend.set_floatx('float64')  # !! Set precision for the entire model here
print("TensorFlow version: {}".format(tf.__version__))
print("Eager execution: {}".format(tf.executing_eagerly()))
print("Num GPUs available: {}".format(len(GPUS)))
print("Training at precision: {}".format(tf.keras.backend.floatx()))
print("Training on device: {}".format(DEVICE))

TensorFlow version: 2.9.2
Eager execution: True
Num GPUs available: 1
Training at precision: float64
Training on device: /GPU:0


In [5]:
# ==============================================================================
# Initialize hyper-parameters and Koopman model
# ==============================================================================
# General Parameters
hyp_params = dict()
hyp_params['sim_start'] = dt.datetime.now().strftime("%Y-%m-%d-%H%M")
hyp_params['experiment'] = 'lorenz96'
hyp_params['plot_path'] = './training_results/' + hyp_params['experiment'] + '_' + hyp_params['sim_start']
hyp_params['model_path'] = './trained_models/' + hyp_params['experiment'] + '_' + hyp_params['sim_start']
hyp_params['device'] = DEVICE
hyp_params['precision'] = tf.keras.backend.floatx()
hyp_params['num_init_conds'] = 15000
hyp_params['num_train_init_conds'] = 10000
hyp_params['num_val_init_conds'] = 3000
hyp_params['num_test_init_conds'] = 2000
hyp_params['time_final'] = 30
hyp_params['delta_t'] = 0.05
hyp_params['num_time_steps'] = int(hyp_params['time_final']/hyp_params['delta_t'])
hyp_params['num_pred_steps'] = hyp_params['num_time_steps']
hyp_params['num_recon_steps'] = hyp_params['num_time_steps'] - 100
hyp_params['max_epochs'] = 100
hyp_params['save_every'] = hyp_params['max_epochs'] // NUM_SAVES
hyp_params['plot_every'] = hyp_params['max_epochs'] // NUM_PLOTS

# Hankel-DMD Window Size and Threshhold
hyp_params['num_observables'] = 1
hyp_params['threshhold'] = 6
hyp_params['observation_dimension'] = 0

# Universal network layer parameters (AE & Aux)
hyp_params['optimizer'] = 'adam'
hyp_params['batch_size'] = 256
hyp_params['phys_dim'] = 5
hyp_params['latent_dim'] = 5
hyp_params['hidden_activation'] = tf.keras.activations.relu
hyp_params['bias_initializer'] = tf.keras.initializers.Zeros


# Encoding/Decoding Layer Parameters
hyp_params['num_en_layers'] = 3
hyp_params['num_en_neurons'] = 128
hyp_params['kernel_init_enc'] = tf.keras.initializers.TruncatedNormal(mean=0.0, stddev=0.1)
hyp_params['kernel_init_dec'] = tf.keras.initializers.TruncatedNormal(mean=0.0, stddev=0.1)
hyp_params['ae_output_activation'] = tf.keras.activations.linear

# Loss Function Parameters
hyp_params['a1'] = tf.constant(1, dtype=hyp_params['precision'])        # Reconstruction
hyp_params['a2'] = tf.constant(1, dtype=hyp_params['precision'])        # DMD
hyp_params['a3'] = tf.constant(1, dtype=hyp_params['precision'])        # Prediction
hyp_params['a4'] = tf.constant(1e-14, dtype=hyp_params['precision'])    # L-2 on weights

# Learning rate
hyp_params['lr'] = 2.5e-3

# Initialize the Koopman model and loss
myMachine = dl.DLDMD(hyp_params)
myLoss = lf.LossDLDMD(hyp_params)

In [6]:
# ==============================================================================
# Generate / load data
# ==============================================================================
data_fname = 'lorenz96_data.pkl'
if os.path.exists(data_fname):
    # Load data from file
    data = pickle.load(open(data_fname, 'rb'))
    data = tf.cast(data, dtype=hyp_params['precision'])

In [7]:
# Create training and validation datasets from the initial conditions
shuffled_data = tf.random.shuffle(data)
ntic = hyp_params['num_train_init_conds']
nvic = hyp_params['num_val_init_conds']
train_data = shuffled_data[:ntic, :, :]
val_data = shuffled_data[ntic:ntic+nvic, :, :]
test_data = shuffled_data[ntic+nvic:, :, :]
pickle.dump(train_data, open('data_train.pkl', 'wb'))
pickle.dump(val_data, open('data_val.pkl', 'wb'))
pickle.dump(test_data, open('data_test.pkl', 'wb'))
train_data = tf.data.Dataset.from_tensor_slices(train_data)
val_data = tf.data.Dataset.from_tensor_slices(val_data)
test_data = tf.data.Dataset.from_tensor_slices(test_data)

# Batch and prefetch the validation data to the GPUs
val_set = val_data.batch(hyp_params['batch_size'], drop_remainder=True)
val_set = val_set.prefetch(tf.data.AUTOTUNE)

In [ ]:
# ==============================================================================
# Train the model
# ==============================================================================
results = tr.train_model(hyp_params=hyp_params, train_data=train_data,
                         val_set=val_set, model=myMachine, loss=myLoss)
print(results['model'].summary())
exit()

Epoch 1 of 100 / Train 1.3915666 / Val 1.1451034 / LR 0.0025000 / 102.51 seconds
Epoch 2 of 100 / Train 1.1874658 / Val 1.0127067 / LR 0.0025000 / 102.43 seconds
Epoch 3 of 100 / Train 1.0814143 / Val 0.8590978 / LR 0.0025000 / 102.43 seconds
Epoch 4 of 100 / Train 1.0645166 / Val 0.7945363 / LR 0.0025000 / 102.43 seconds
Epoch 5 of 100 / Train 0.9845983 / Val 0.6896327 / LR 0.0025000 / 102.42 seconds
Epoch 6 of 100 / Train 0.8537387 / Val 0.5330357 / LR 0.0025000 / 102.42 seconds
Epoch 7 of 100 / Train 0.7875675 / Val 0.5012915 / LR 0.0025000 / 102.43 seconds
Epoch 8 of 100 / Train 0.6745438 / Val 0.4720855 / LR 0.0025000 / 102.43 seconds
Epoch 9 of 100 / Train 0.6002384 / Val 0.3821852 / LR 0.0025000 / 102.42 seconds
Epoch 10 of 100 / Train 0.5989442 / Val 0.3692463 / LR 0.0025000 / 102.43 seconds
Epoch 11 of 100 / Train 0.5535171 / Val 0.2955941 / LR 0.0025000 / 102.43 seconds
Epoch 12 of 100 / Train 0.5522763 / Val 0.3149118 / LR 0.0025000 / 93.22 seconds
Epoch 13 of 100 / Train 0.